In [8]:
import os
import pandas as pd
from dammit.fileio.gff3 import GFF3Parser

/opt/miniconda3/envs/py3.dammit/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/opt/miniconda3/envs/py3.dammit/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
assemblies_dir = "assemblies/"
assemblies = os.listdir(assemblies_dir)
annotations_dir = "gff3_fix/"
gff3_files = os.listdir(annotations_dir)
print(len(gff3_files))

17


In [3]:
print(gff3_files)

['L_parva.trinity_out.Trinity.fasta.dammit.gff3', 'F_sciadicus.trinity_out.Trinity.fasta.dammit.gff3', 'F_zebrinus.trinity_out.Trinity.fasta.dammit.gff3', 'F_olivaceous.trinity_out.Trinity.fasta.dammit.gff3', 'F_rathbuni.trinity_out.Trinity.fasta.dammit.gff3', 'L_goodei.trinity_out.Trinity.fasta.dammit.gff3', 'F_grandis.trinity_out.Trinity.fasta.dammit.gff3', 'F_chrysotus.trinity_out.Trinity.fasta.dammit.gff3', 'F_parvapinis.trinity_out.Trinity.fasta.dammit.gff3', 'F_notatus.trinity_out.Trinity.fasta.dammit.gff3', 'F_diaphanus.trinity_out.Trinity.fasta.dammit.gff3', 'F_heteroclitusMDPP.trinity_out.Trinity.fasta.dammit.gff3', 'F_similis.trinity_out.Trinity.fasta.dammit.gff3', 'F_heteroclitusMDPL.trinity_out.Trinity.fasta.dammit.gff3', 'A_xenica.trinity_out.Trinity.fasta.dammit.gff3', 'F_catanatus.trinity_out.Trinity.fasta.dammit.gff3', 'F_notti.trinity_out.Trinity.fasta.dammit.gff3']


In [5]:
annotations.columns

Index(['Dbxref', 'ID', 'Name', 'Note', 'Parent', 'Target', 'accuracy',
       'bitscore', 'database', 'end', 'env_coords', 'phase', 'score', 'seqid',
       'source', 'start', 'strand', 'trunc', 'type'],
      dtype='object')

In [16]:
annotations_dir = "gff3_fix/"
gff3_files = os.listdir(annotations_dir)
Trinity_dammit_conversions_dir = "namemaps/reformatted/"
for gff3_file in gff3_files:
    print(gff3_file)
    species = gff3_file.split(".")[0]
    conversion_file = Trinity_dammit_conversions_dir + species + ".dammit.Trinity.conversions.csv"
    conversions = pd.read_csv(conversion_file)
    conversions = conversions.drop(columns=['Unnamed: 0'])
    conversions.columns = ['seqid','TrinityID']
    print("conversions:",conversions.shape)
    gff3_file = annotations_dir + gff3_file
    annotations = GFF3Parser(filename=gff3_file).read()
    print("parsed annotations:",annotations.shape)
    # keeps track of how long the annotation is
    annotations["length"] = annotations["end"].subtract(annotations["start"], fill_value=0)
    # make new table for each with seqid, Name, start, end, length 
    # NCBI
    ncbi_fhet = annotations.loc[annotations['database'] == "protein.fa"]
    ncbi_fhet = ncbi_fhet.sort_values(by=['seqid','length'],ascending=False).drop_duplicates(subset='seqid')[['seqid', 'Name','start','end','length']]
    ncbi_fhet = ncbi_fhet.rename(columns = {'Name':'NCBI'})
    print('NCBI annotations',ncbi_fhet.shape)
    # evigene
    evigene_fhet = annotations.loc[annotations['database'] == "kfish2rae5g.pub.aa"]
    evigene_fhet = evigene_fhet.sort_values(by=['seqid','length'],ascending=False).drop_duplicates(subset='seqid')[['seqid', 'Name','start','end','length']]
    evigene_fhet = evigene_fhet.rename(columns = {'Name':'EviGene'})
    print('evigene annotations',evigene_fhet.shape)
    # Ensembl
    ensembl_fhet = annotations.loc[annotations['database'] == "Fundulus_heteroclitus.Fundulus_heteroclitus-3.0.2.pep.all.fa"]
    ensembl_fhet = ensembl_fhet.sort_values(by=['seqid','length'],ascending=False).drop_duplicates(subset='seqid')[['seqid', 'Name','start','end','length']]
    ensembl_fhet = ensembl_fhet.rename(columns = {'Name':'Ensembl'}) 
    print('ensembl annotations',ensembl_fhet.shape)
    all_annotations = pd.merge(ncbi_fhet,evigene_fhet,on='seqid',how='outer')
    print('merging ncbi_fhet,evigene',all_annotations.shape)
    all_annotations = pd.merge(ensembl_fhet,all_annotations,on='seqid',how='outer')
    print('merging with ensembl',all_annotations.shape)
    all_annotations = pd.merge(conversions,all_annotations,on='seqid',how='outer')
    print('merging with Trinity conversions',all_annotations.shape)
    annotations_file = "annotations_conversions/" + species + ".annotations.conversions.csv"
    all_annotations.to_csv(annotations_file)
    print("Written:",annotations_file)

L_parva.trinity_out.Trinity.fasta.dammit.gff3
conversions: (341080, 2)
parsed annotations: (2836988, 19)
NCBI annotations (75896, 5)
evigene annotations (114509, 5)
ensembl annotations (77499, 5)
merging ncbi_fhet,evigene (115899, 9)
merging with ensembl (117160, 13)
merging with Trinity conversions (341080, 14)
Written: annotations_conversions/L_parva.annotations.conversions.csv
F_sciadicus.trinity_out.Trinity.fasta.dammit.gff3
conversions: (199646, 2)
parsed annotations: (1760395, 19)
NCBI annotations (50139, 5)
evigene annotations (75228, 5)
ensembl annotations (51556, 5)
merging ncbi_fhet,evigene (76023, 9)
merging with ensembl (76848, 13)
merging with Trinity conversions (199646, 14)
Written: annotations_conversions/F_sciadicus.annotations.conversions.csv
F_zebrinus.trinity_out.Trinity.fasta.dammit.gff3
conversions: (230099, 2)
parsed annotations: (1959514, 19)
NCBI annotations (57238, 5)
evigene annotations (87269, 5)
ensembl annotations (60805, 5)
merging ncbi_fhet,evigene (8815

In [6]:
annotations.head(50)

,Dbxref,ID,Name,Note,Parent,Target,accuracy,bitscore,database,end,env_coords,phase,score,seqid,source,start,strand,trunc,type
0,"""Pfam:PF01777.14""",homology:299683,Ribosomal_L27e,Ribosomal L27e protein family,NaN,Ribosomal_L27e 1 85 +,1.0,NaN,NaN,408,154 408,NaN,1.200000e-38,Transcript_0,HMMER,153,NaN,NaN,protein_hmm_match
1,"""Pfam:PF00467.25""",homology:299684,KOW,KOW motif,NaN,KOW 1 27 +,0.88,NaN,NaN,105,19 120,NaN,3.600000e-07,Transcript_0,HMMER,18,NaN,NaN,protein_hmm_match
2,NaN,homology:418357,sp|Q7ZV82|RL27_DANRE,NaN,NaN,sp|Q7ZV82|RL27_DANRE 0 136 +,NaN,NaN,sprot,460,NaN,NaN,2.300000e-116,Transcript_0,LAST,52,-,NaN,translated_nucleotide_match
3,NaN,homology:327537,ENSXMAP00000014064,NaN,NaN,ENSXMAP00000014064 0 136 +,NaN,NaN,OrthoDB,460,NaN,NaN,3.800000e-117,Transcript_0,LAST,52,-,NaN,translated_nucleotide_match
4,NaN,homology:491304,ref|XP_012707384.1| 60S ribosomal protein L27 ...,NaN,NaN,ref|XP_012707384.1| 60S ribosomal protein L27 ...,NaN,NaN,protein.fa,153,NaN,NaN,5.100000e-97,Transcript_0,shmlast.LAST,17,+,NaN,conditional_reciprocal_best_LAST
5,NaN,homology:616254,Funhe2EKm006123t1,NaN,NaN,Funhe2EKm006123t1 20 149 +,NaN,NaN,kfish2rae5g.pub.aa,153,NaN,NaN,1.300000e-90,Transcript_0,shmlast.LAST,24,+,NaN,conditional_reciprocal_best_LAST
6,NaN,homology:616252,Funhe2EKm006123t4,NaN,NaN,Funhe2EKm006123t4 0 49 +,NaN,NaN,kfish2rae5g.pub.aa,80,NaN,NaN,1.400000e-21,Transcript_0,shmlast.LAST,31,+,NaN,conditional_reciprocal_best_LAST
7,NaN,homology:616249,Funhe2EKm006123t6,NaN,NaN,Funhe2EKm006123t6 14 161 +,NaN,NaN,kfish2rae5g.pub.aa,151,NaN,NaN,9.100000e-93,Transcript_0,shmlast.LAST,4,+,NaN,conditional_reciprocal_best_LAST
8,NaN,homology:1061661,ENSFHEP00000005448,NaN,NaN,ENSFHEP00000005448 16 113 +,NaN,NaN,Fundulus_heteroclitus.Fundulus_heteroclitus-3....,153,NaN,NaN,4.400000e-66,Transcript_0,shmlast.LAST,56,+,NaN,conditional_reciprocal_best_LAST
9,NaN,cds.Gene.1::Transcript_0::g.1::m.1,NaN,NaN,Gene.1::Transcript_0::g.1::m.1,NaN,NaN,NaN,NaN,496,NaN,0.0,NaN,Transcript_0,transdecoder,85,-,NaN,CDS


# Testing code (do not use)

In [17]:
annotations = GFF3Parser(filename="A_xenica.trinity_out.Trinity.fasta.dammit.gff3").read()

/opt/miniconda3/envs/dammit_working_gff3/lib/python3.6/site-packages/dammit-1.1-py3.6.egg/dammit/fileio/base.py:79: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat(self, ignore_index=True)


In [18]:
annotations.head()

,Dbxref,ID,Name,Note,Parent,Target,accuracy,bitscore,database,end,env_coords,phase,score,seqid,source,start,strand,trunc,type
0,NaN,homology:425062,90820,NaN,NaN,90820 1132 1240 +,NaN,NaN,OrthoDB,366,NaN,NaN,5.500000e-47,Transcript_100000,LAST,36,+,NaN,translated_nucleotide_match
1,"""Pfam:PF13465.2""",homology:53130,zf-H2C2_2,Zinc-finger double domain,NaN,zf-H2C2_2 1 25 +,0.94,NaN,NaN,198,127 201,NaN,1.200000e-03,Transcript_100001,HMMER,126,NaN,NaN,protein_hmm_match
2,"""Pfam:PF13894.2""",homology:53126,zf-C2H2_4,C2H2-type zinc finger,NaN,zf-C2H2_4 2 23 +,0.94,NaN,NaN,234,166 237,NaN,4.000000e-03,Transcript_100001,HMMER,168,NaN,NaN,protein_hmm_match
3,"""Pfam:PF12171.4""",homology:53128,zf-C2H2_jaz,Zinc-finger double-stranded RNA-binding,NaN,zf-C2H2_jaz 4 23 +,0.93,NaN,NaN,231,172 240,NaN,2.600000e-03,Transcript_100001,HMMER,171,NaN,NaN,protein_hmm_match
4,"""Pfam:PF13912.2""",homology:53133,zf-C2H2_6,C2H2-type zinc finger,NaN,zf-C2H2_6 4 24 +,0.96,NaN,NaN,234,169 240,NaN,5.800000e-03,Transcript_100001,HMMER,171,NaN,NaN,protein_hmm_match
